# Produce tables of stage 2 MWAS results

In [1]:
library(data.table)
library(foreach)
library(doParallel)

Loading required package: iterators

Loading required package: parallel



In [2]:
getwd()

[1] "/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/scripts"

In [3]:
df <- fread("12-OUT_matched_SNP_meth_cov_outputs.csv")

In [4]:
print(nrow(df))

[1] 16098


In [5]:
library(data.table)
library(ggplot2)

In [6]:
# Initialize logging
log_file <- file("processing_log.txt", open = "wt")
sink(log_file, type = "message")
sink(log_file, type = "output", append = TRUE)

In [7]:
traits <- c("bp", "mdd", "scz")
df$stage2_paths <- gsub(".rds", "_gwas_stat_", df$path)
df$final_paths <- vector("list", length(df$stage2_paths))

In [8]:
for (trait in traits) {
  message("Processing trait: ", trait)
  df$final_paths <- paste0(df$stage2_paths, trait, "_results.rds")
}

Processing trait: bp

Processing trait: mdd

Processing trait: scz



In [ ]:
library(parallel)
library(data.table)

# Function to process a chunk of files with detailed logging
process_chunk <- function(trait, indices) {
  results <- lapply(indices, function(i) {
    tryCatch({
      if (grepl("empty", df$final_paths[i])) return(NULL)
      stage2_in <- readRDS(df$final_paths[i])
      stage1_in <- readRDS(df$path[i])

      if (length(stage1_in@models) != length(stage2_in@MWASmodels)) {
        stop("Files don't match")
      }

      data_list <- vector("list", length(stage1_in@models))
      for (j in seq_along(stage1_in@models)) {
        model1 <- stage1_in@models[[j]]
        model2 <- stage2_in@MWASmodels[[j]]

        data_list[[j]] <- data.table(
          z = model2["z"],
          p = model2["p"],
          n = model2["n"],
          chr = df$Chr[i],
          pos = model1@methylationPosition,
          population = df$population[i],
          region = df$region[i],
          stats = stage2_in@summary_stats_path,
          scaff = stage1_in@scaffoldIdentifier
        )
      }

      result <- rbindlist(data_list, use.names = TRUE, fill = TRUE)
      result
    }, error = function(e) {
      message("Error processing file index ", i, ": ", e$message)
      list(error = e$message, index = i, path = df$final_paths[i])
    })
  })

  problematic_item_index <- which(sapply(results, function(x) is.list(x) && "error" %in% names(x)))

  if (length(problematic_item_index) > 0) {
    message("Problematic item found at index: ", problematic_item_index)
    for (index in problematic_item_index) {
      message("Details for problematic item at index ", index, ":")
      print(results[[index]])
      print(head(df[index, ]))
      message("Shape of problematic item: ", nrow(df[index, ]), " rows, ", ncol(df[index, ]), " columns")
    }
  } else {
    message("No problematic items found.")
  }

  rbindlist(results[sapply(results, is.data.table)], use.names = TRUE, fill = TRUE)
}

# Function to process a single trait
process_trait <- function(trait) {
  message("Processing trait: ", trait)
  df$final_paths <- paste0(df$stage2_paths, trait, "_results.rds")

  n <- length(df$final_paths)
  chunk_size <- 50  # Set a reasonable chunk size
  chunk_indices <- split(seq_len(n), ceiling(seq_len(n) / chunk_size))

  # Process chunks in parallel
  combined_results <- mclapply(chunk_indices, function(indices) process_chunk(trait, indices), mc.cores = 125)
  combined_results <- rbindlist(combined_results, use.names = TRUE, fill = TRUE)

  # Write the combined results at the end
  output_file <- paste0("16a9par-OUT_stage2_MWAS_", trait, ".csv")
  fwrite(combined_results, output_file)
}

# Sample a subset of the files to estimate runtime
sample_files <- function(trait, sample_size = 50) {
  df$final_paths <- paste0(df$stage2_paths, trait, "_results.rds")
  sampled_indices <- sample(seq_along(df$final_paths), sample_size)

  # Process sample chunks in parallel
  combined_sample_results <- mclapply(sampled_indices, function(i) process_chunk(trait, list(i)), mc.cores = 125)
  combined_sample_results <- rbindlist(combined_sample_results, use.names = TRUE, fill = TRUE)

  combined_sample_results
}

# Estimate runtime for a single trait with sampled files
estimate_runtime <- function(trait, sample_size = 50) {
  start_time <- Sys.time()
  sample_results <- sample_files(trait, sample_size)
  end_time <- Sys.time()
  time_taken <- as.numeric(difftime(end_time, start_time, units = "secs"))
  estimated_total_time <- time_taken * (length(df$final_paths) / sample_size)
  message("Estimated time for trait ", trait, ": ", estimated_total_time, " seconds")
  estimated_total_time
}

# Print estimated runtimes for all traits
estimated_times <- sapply(traits, estimate_runtime)

# Log estimated runtimes
log_file <- "estimated_runtimes.log"
writeLines(paste("Estimated runtimes for traits:", paste(traits, estimated_times, sep = ": ", collapse = "\n")), log_file)

# Process all traits and print speeds to a file
log_file <- "processing_times.log"
file_conn <- file(log_file, open = "wt")
sink(file = file_conn, type = "message")
sink(file = file_conn, type = "output")

start_time <- Sys.time()

# Use lapply to process traits sequentially to ensure all traits are processed
results <- lapply(traits, function(trait) {
  tryCatch({
    process_trait(trait)
  }, error = function(e) {
    message("Error processing trait ", trait, ": ", e$message)
  })
})

end_time <- Sys.time()

total_time <- as.numeric(difftime(end_time, start_time, units = "secs"))
message("Total processing time: ", total_time, " seconds")

sink(type = "message")
sink(type = "output")
close(file_conn)


Estimated time for trait bp: 3018.31013674736 seconds

Estimated time for trait mdd: 2558.44264818192 seconds

Estimated time for trait scz: 2263.79644872665 seconds

Processing trait: bp



In [9]:
getwd()

[1] "/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/scripts"

## Trust but verify

In [ ]:
library(parallel)
library(data.table)

# Function to process a chunk of files with detailed logging
process_chunk <- function(trait, indices) {
  results <- lapply(indices, function(i) {
    tryCatch({
      if (grepl("empty", df$final_paths[i])) return(NULL)
      stage2_in <- readRDS(df$final_paths[i])
      stage1_in <- readRDS(df$path[i])

      if (length(stage1_in@models) != length(stage2_in@MWASmodels)) {
        stop("Files don't match")
      }

      data_list <- vector("list", length(stage1_in@models))
      for (j in seq_along(stage1_in@models)) {
        model1 <- stage1_in@models[[j]]
        model2 <- stage2_in@MWASmodels[[j]]

        data_list[[j]] <- data.table(
          z = model2["z"],
          p = model2["p"],
          n = model2["n"],
          pos = model1@methylationPosition,
          stats = stage2_in@summary_stats_path,
          scaff = stage1_in@scaffoldIdentifier
        )
      }

      result <- rbindlist(data_list, use.names = TRUE, fill = TRUE)
      result
    }, error = function(e) {
      message("Error processing file index ", i, ": ", e$message)
      list(error = e$message, index = i, path = df$final_paths[i])
    })
  })

  problematic_item_index <- which(sapply(results, function(x) is.list(x) && "error" %in% names(x)))

  if (length(problematic_item_index) > 0) {
    message("Problematic item found at index: ", problematic_item_index)
    for (index in problematic_item_index) {
      message("Details for problematic item at index ", index, ":")
      print(results[[index]])
      print(head(df[index, ]))
    }
  } else {
    message("No problematic items found.")
  }

  rbindlist(results[sapply(results, is.data.table)], use.names = TRUE, fill = TRUE)
}

# Function to process a single trait
process_trait <- function(trait) {
  message("Processing trait: ", trait)
  df$final_paths <- paste0(df$stage2_paths, trait, "_results.rds")

  n <- length(df$final_paths)
  chunk_size <- 50  # Set a reasonable chunk size
  chunk_indices <- split(seq_len(n), ceiling(seq_len(n) / chunk_size))

  # Process chunks sequentially (no parallelization for test)
  combined_results <- lapply(chunk_indices, function(indices) process_chunk(trait, indices))
  combined_results <- rbindlist(combined_results, use.names = TRUE, fill = TRUE)

  # Write the combined results at the end
  output_file <- paste0("test-16a7par-OUT_stage2_MWAS_", trait, ".csv")
  fwrite(combined_results, output_file)
}

# Function to run the test for multiple traits
test_multiple_traits <- function(traits) {
  # Use a small subset of df for testing
  df_subset <- df[1:100, ]

  # Process each trait
  lapply(traits, function(trait) {
    process_trait(trait)
  })
}

# Run the test for a small subset of the data for multiple traits
test_traits <- traits[1:3]  # Use the first three traits for testing
test_multiple_traits(test_traits)


Why the error?

In [13]:
library(parallel)
library(data.table)

# Function to process a chunk of files with detailed logging
process_chunk <- function(trait, indices) {
  results <- lapply(indices, function(i) {
    tryCatch({
      if (grepl("empty", df$final_paths[i])) return(NULL)
      stage2_in <- readRDS(df$final_paths[i])
      stage1_in <- readRDS(df$path[i])

      if (length(stage1_in@models) != length(stage2_in@MWASmodels)) {
        stop("Files don't match")
      }

      data_list <- vector("list", length(stage1_in@models))
      for (j in seq_along(stage1_in@models)) {
        model1 <- stage1_in@models[[j]]
        model2 <- stage2_in@MWASmodels[[j]]

        data_list[[j]] <- data.table(
          z = model2["z"],
          p = model2["p"],
          n = model2["n"],
          pos = model1@methylationPosition,
          stats = stage2_in@summary_stats_path,
          scaff = stage1_in@scaffoldIdentifier
        )
      }

      result <- rbindlist(data_list, use.names = TRUE, fill = TRUE)
      result
    }, error = function(e) {
      message("Error processing file index ", i, ": ", e$message)
      NULL
    })
  })

  # Identify problematic items
  problematic_item_index <- which(sapply(results, function(x) !is.data.table(x) && !is.list(x) && !is.null(x)))
  
  if (length(problematic_item_index) > 0) {
    message("Problematic item found at index: ", problematic_item_index)
    message("Corresponding final_path: ", df$final_paths[problematic_item_index])
  } else {
    message("No problematic items found.")
  }
  
  results  # Return the list of results directly for debugging
}

# Function to sample files and diagnose issues
sample_files_diagnostic <- function(trait, sample_size = 50) {
  df$final_paths <- paste0(df$stage2_paths, trait, "_results.rds")
  sampled_indices <- sample(seq_along(df$final_paths), sample_size)
  
  # Process sample chunks in parallel
  combined_sample_results <- mclapply(sampled_indices, function(i) process_chunk(trait, list(i)), mc.cores = 125)

  combined_sample_results
}

# Use one of the traits for a diagnostic run
trait <- traits[1]  # Replace with a specific trait if needed
sample_files_diagnostic(trait, sample_size = 50)


z,p,n,pos,stats,scaff
<dbl>,<dbl>,<dbl>,<int>,<chr>,<chr>
NA,NA,1,143788669,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr3-chr3_AA-libd_chr3-chr3_AA-1120001-1140000-dynamic-1corestotal-allcorepera-20240513-171916
NA,NA,1,143789071,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr3-chr3_AA-libd_chr3-chr3_AA-1120001-1140000-dynamic-1corestotal-allcorepera-20240513-171916
NA,NA,1,143789116,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr3-chr3_AA-libd_chr3-chr3_AA-1120001-1140000-dynamic-1corestotal-allcorepera-20240513-171916
NA,NA,1,143789219,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr3-chr3_AA-libd_chr3-chr3_AA-1120001-1140000-dynamic-1corestotal-allcorepera-20240513-171916
NA,NA,1,143789289,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr3-chr3_AA-libd_chr3-chr3_AA-1120001-1140000-dynamic-1corestotal-allcorepera-20240513-171916
NA,NA,1,143789932,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr3-chr3_AA-libd_chr3-chr3_AA-1120001-1140000-dynamic-1corestotal-allcorepera-20240513-171916
0.001946240,0.9984471,3,143790141,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr3-chr3_AA-libd_chr3-chr3_AA-1120001-1140000-dynamic-1corestotal-allcorepera-20240513-171916
0.002176963,0.9982630,2,143790162,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr3-chr3_AA-libd_chr3-chr3_AA-1120001-1140000-dynamic-1corestotal-allcorepera-20240513-171916
0.002002436,0.9984023,2,143790223,/expanse/lustre/projects/jhu152/naglemi/mwas/gwas/gwas_stat_scz,libd_chr3-chr3_AA-libd_chr3-chr3_AA-1120001-1140000-dynamic-1corestotal-allcorepera-20240513-171916


In [11]:
print(sampled_indices[25])
print(df$final_paths[sampled_indices[25]])
print(df$path[sampled_indices[25]])
print(process_chunk(trait, list(sampled_indices[25])))


ERROR: Error in eval(expr, envir, enclos): object 'sampled_indices' not found


In [12]:
which(df$final_paths == "..//output_EXPANSE_a2_hippo/libd_chr2-chr2_all-libd_chr2-chr2_all-1520001-1540000-dynamic-1corestotal-allcorepera-20240505-001208_gwas_stat_bp_results.rds")

integer(0)

## Investigate the files that didn't finish and repeat if needed

In [14]:
dim(df)

[1] 16098    16

In [15]:
df$final_paths[4294]

[1] "..//output_EXPANSE_a2_hippo/libd_chr2-chr2_all-libd_chr2-chr2_all-1520001-1540000-dynamic-1corestotal-allcorepera-20240505-001208_gwas_stat_scz_results.rds"

In [16]:
file.exists(df$final_paths[4294])

[1] TRUE

In [17]:
file.exists(df$final_paths[4293])

[1] TRUE

In [18]:
file.exists(df$final_paths[4299])

[1] TRUE

In [19]:
df$exists <- file.exists(df$final_paths)

In [20]:
levels(factor(df$exists))

[1] "FALSE" "TRUE"

In [21]:
table(df$exists)


FALSE  TRUE 
   54 16044 

In [5]:
getwd()

[1] "/expanse/lustre/projects/jhu152/naglemi/mwas/CpGWAS/scripts"

## Checking logs etc

In [1]:
# Function to check for errors in the log file
check_log_for_errors <- function(log_path) {
  log_contents <- readLines(log_path)
  errors <- grep("error|stop|interrupted", tolower(log_contents), value = TRUE)
  if (length(errors) > 0) {
    return(list(success = FALSE, messages = errors))
  }
  return(list(success = TRUE))
}

# Usage
log_result <- check_log_for_errors("processing_log.txt")
if (log_result$success) {
  cat("No errors found in log.\n")
} else {
  cat("Errors found in log:\n", paste(log_result$messages, collapse = "\n"))
}


No errors found in log.


In [1]:
library(data.table)

In [4]:
# SLOW Function to validate output files
validate_output_files <- function(trait_names, output_path_template) {
  for (trait in trait_names) {
    file_path <- sprintf(output_path_template, trait)
    if (!file.exists(file_path)) {
      cat(sprintf("Output file for %s does not exist.\n", trait))
      next
    }
    dt <- fread(file_path)
    if (nrow(dt) == 0) {
      cat(sprintf("Output file for %s is empty.\n", trait))
    } else {
      cat(sprintf("Output file for %s has %d rows.\n", trait, nrow(dt)))
    }
  }
}

# Usage
#validate_output_files(c("bp", "mdd", "scz"), "16a5-OUT_stage2_MWAS_%s.csv")


In [2]:
# Function to check if output files exist and are not empty
check_output_files_existence_and_size <- function(trait_names, output_path_template) {
  results <- list()
  for (trait in trait_names) {
    file_path <- sprintf(output_path_template, trait)
    if (!file.exists(file_path)) {
      results[[trait]] <- sprintf("Output file for %s does not exist.", trait)
    } else {
      file_size <- file.info(file_path)$size
      if (file_size > 0) {
        results[[trait]] <- sprintf("Output file for %s is valid with size %d bytes.", trait, file_size)
      } else {
        results[[trait]] <- sprintf("Output file for %s exists but is empty.", trait)
      }
    }
  }
  return(results)
}

# Usage
file_check_results <- check_output_files_existence_and_size(c("bp", "mdd", "scz"), "16a5-OUT_stage2_MWAS_%s.csv")
for (result in file_check_results) {
  cat(result, "\n")
}


ERROR: Error in sprintf("Output file for %s is valid with size %d bytes.", trait, : invalid format '%d'; use format %f, %e, %g or %a for numeric objects


In [3]:
# Function to check if output files exist and are not empty
check_output_files_existence_and_size <- function(trait_names, output_path_template) {
  results <- list()
  for (trait in trait_names) {
    file_path <- sprintf(output_path_template, trait)
    if (!file.exists(file_path)) {
      results[[trait]] <- sprintf("Output file for %s does not exist.", trait)
    } else {
      file_size <- as.integer(file.info(file_path)$size)  # Ensure integer format
      if (file_size > 0) {
        results[[trait]] <- sprintf("Output file for %s is valid with size %d bytes.", trait, file_size)
      } else {
        results[[trait]] <- sprintf("Output file for %s exists but is empty.", trait)
      }
    }
  }
  return(results)
}

# Usage
file_check_results <- check_output_files_existence_and_size(c("bp", "mdd", "scz"), "16a5-OUT_stage2_MWAS_%s.csv")
for (result in file_check_results) {
  cat(result, "\n")
}


Warning message in check_output_files_existence_and_size(c("bp", "mdd", "scz"), :
“NAs introduced by coercion to integer range”


ERROR: Error in if (file_size > 0) {: missing value where TRUE/FALSE needed


In [4]:
# Function to check if output files exist and are not empty
check_output_files_existence_and_size <- function(trait_names, output_path_template) {
  results <- list()
  for (trait in trait_names) {
    file_path <- sprintf(output_path_template, trait)
    if (!file.exists(file_path)) {
      results[[trait]] <- sprintf("Output file for %s does not exist.", trait)
    } else {
      file_size <- file.info(file_path)$size  # Get the file size without casting
      if (!is.na(file_size) && file_size > 0) {
        results[[trait]] <- sprintf("Output file for %s is valid with size %f bytes.", trait, file_size)
      } else if (is.na(file_size)) {
        results[[trait]] <- sprintf("Error retrieving file size for %s.", trait)
      } else {
        results[[trait]] <- sprintf("Output file for %s exists but is empty.", trait)
      }
    }
  }
  return(results)
}

# Usage
file_check_results <- check_output_files_existence_and_size(c("bp", "mdd", "scz"), "16a5-OUT_stage2_MWAS_%s.csv")
for (result in file_check_results) {
  cat(result, "\n")
}


Output file for bp is valid with size 16912150884.000000 bytes. 
Output file for mdd does not exist. 
Output file for scz does not exist. 


## Second attempt

In [6]:
library(data.table)
library(ggplot2)

# Initialize logging
log_file <- file("processing_log.txt", open = "wt")
sink(log_file, type = "message")
sink(log_file, type = "output", append = TRUE)

tryCatch({
    traits <- c("bp", "mdd", "scz")
    df$stage2_paths <- gsub(".rds", "_gwas_stat_", df$path)
    df$final_paths <- vector("list", length(df$stage2_paths))

    for (trait in traits) {
        message("Processing trait: ", trait)
        df$final_paths <- paste0(df$stage2_paths, trait, "_results.rds")
        output_file <- paste0("16a6-OUT_stage2_MWAS_", trait, ".csv")
        header_written <- FALSE

        for (i in seq_along(df$final_paths)) {
            if (grepl("empty", df$final_paths[i])) next
            
            message("Processing file ", i, " of ", length(df$final_paths))
            stage2_in <- readRDS(df$final_paths[i])
            stage1_in <- readRDS(df$path[i])
            
            if (length(stage1_in@models) != length(stage2_in@MWASmodels)) {
                stop("Files don't match")
            }

            data_list <- vector("list", length(stage1_in@models))
            for (j in seq_along(stage1_in@models)) {
                model1 <- stage1_in@models[[j]]
                model2 <- stage2_in@MWASmodels[[j]]

                data_list[[j]] <- data.table(
                    z = model2["z"],
                    p = model2["p"],
                    n = model2["n"],
                    pos = model1@methylationPosition,
                    stats = stage2_in@summary_stats_path,
                    scaff = stage1_in@scaffoldIdentifier
                )
            }

            combined_data <- rbindlist(data_list, use.names = TRUE, fill = TRUE)

            # Write data incrementally
            if (!header_written) {
                fwrite(combined_data, output_file)
                header_written <- TRUE
            } else {
                fwrite(combined_data, output_file, append = TRUE)
            }
        }
    }
}, error = function(e) {
    message("An error occurred: ", e$message)
}, finally = {
    sink(NULL)  # Turn off logging
    close(log_file)
    message("Logging ended.")
})



An error occurred: object of type 'closure' is not subsettable



ERROR: Error in close.connection(log_file): cannot close 'message' sink connection
